In [ ]:
import pandas as pd

def modify_ply(input_file, output_file):
    """
    Converts all occurrences of scalar_Classification value 5 to 1 (i.e. barrier class to building class)
    """
    
    # Read the .ply file as DataFrame, skipping the header
    with open(input_file, 'r') as file:
        lines = file.readlines()
    header_end = lines.index('end_header\n') + 1
    data = pd.read_csv(input_file, sep=' ', header=None, skiprows=header_end)
    
    # Modify the scalar_Classification column where it's 5
    data[3] = data[3].replace(5.0, 1.0)
    
    # Write the header and modified data back to a new file
    with open(output_file, 'w') as file:
        file.writelines(lines[:header_end])  # Write header
        data.to_csv(file, header=False, index=False, sep=' ', float_format='%.6f')

# Example usage
modify_ply('A1-A10_ongoing4.ply', 'A1-A10_ongoing5.ply')


In [1]:
!ls

A1-A10_ongoing3.ply  A1-A10_ongoing4.ply  Untitled.ipynb


In [ ]:
import os

def process_ply_file(input_filename, output_filename, classification_to_remove=12.0):
    """
    Reads an ASCII PLY file, performs multiple cleaning and reformatting operations,
    and writes the result to a new file.

    Operations:
    1. Remove points with duplicate (x, y, z) coordinates.
    2. Remove points where scalar_Classification is equal to the specified value (default 12.0).
    3. Remove the 'scalar_Original_cloud_index' property from the header and data.
    4. Rename 'scalar_Classification' to 'class' and change its type from float to int.
    """

    print(f"Reading from: {input_filename}")
    print(f"Writing to: {output_filename}")
    print(f"Removing points with scalar_Classification = {classification_to_remove}...")
    
    if not os.path.exists(input_filename):
        print(f"Error: Input file not found at '{input_filename}'")
        return

    # --- Initializing Counters and Storage ---
    header_lines = []
    points = []
    
    # Store unique coordinates to detect duplicates
    unique_coords = set()
    
    # Track the indices of the properties we want to KEEP
    x_idx, y_idx, z_idx = -1, -1, -1
    classification_idx = -1
    
    # Final data list after filtering
    processed_points = []

    # --- 1. Read and Parse the Input File ---
    try:
        with open(input_filename, 'r') as infile:
            in_header = True
            
            for line in infile:
                line = line.strip()
                if not line:
                    continue

                if in_header:
                    header_lines.append(line)
                    if line == 'end_header':
                        in_header = False
                    continue
                
                # --- Read Point Data ---
                parts = line.split()
                if len(parts) >= 5: # Expecting at least x, y, z, classification, index
                    points.append(parts)

    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return
        
    # --- 2. Process Header to Determine Indices and Create New Header ---
    
    new_header = []
    property_names = []
    
    for line in header_lines:
        new_line = line
        
        # 2a. Remove 'scalar_Original_cloud_index' and adjust 'scalar_Classification'
        if line.startswith('property'):
            parts = line.split()
            prop_type = parts[1]
            prop_name = parts[2]
            property_names.append(prop_name)
            
            if prop_name == 'scalar_Original_cloud_index':
                # Skip this property
                continue
            elif prop_name == 'scalar_Classification':
                # Rename and change type
                new_line = f"property int class"
                # Note: Index determination will be done later based on property_names
            
            elif prop_name in ['x', 'y', 'z']:
                # Track original property indices
                if prop_name == 'x': x_idx = len(property_names) - 1
                if prop_name == 'y': y_idx = len(property_names) - 1
                if prop_name == 'z': z_idx = len(property_names) - 1
                
            new_header.append(new_line)
            
        elif line.startswith('element vertex'):
            # This line will be updated with the final count later
            new_header.append(line)
        else:
            # Keep other header lines (ply, format, comments, end_header)
            new_header.append(line)
            
    # Find the original classification index after tracking all properties
    if 'scalar_Classification' in property_names:
        classification_idx = property_names.index('scalar_Classification')

    if x_idx == -1 or y_idx == -1 or z_idx == -1 or classification_idx == -1:
        print("Error: Required properties (x, y, z, scalar_Classification) not found in header.")
        return

    # --- 3. Filter and Process Points ---
    
    points_removed_duplicate = 0
    points_removed_classification = 0
    
    for parts in points:
        try:
            # Get coordinates for duplication check
            x = parts[x_idx]
            y = parts[y_idx]
            z = parts[z_idx]
            
            # Key for duplicate check (coordinates)
            coord_key = (x, y, z)
            
            # 3a. Check for Duplicates
            if coord_key in unique_coords:
                points_removed_duplicate += 1
                continue
            
            # 3b. Check for Classification
            current_classification = float(parts[classification_idx])
            if current_classification == classification_to_remove:
                points_removed_classification += 1
                continue
            
            # Add to unique set
            unique_coords.add(coord_key)
            
            # --- 3c. Modify and Format Point ---
            new_parts = []
            
            # Track the original index in the input data
            original_index = 0
            
            for i, val in enumerate(parts):
                if i == x_idx or i == y_idx or i == z_idx:
                    # Keep x, y, z
                    new_parts.append(val)
                elif i == classification_idx:
                    # Change type from float to int and rename/reformat
                    int_class = int(round(current_classification))
                    new_parts.append(str(int_class)) # 'class' property, now int
                elif property_names[i] == 'scalar_Original_cloud_index':
                    # Skip 'scalar_Original_cloud_index' values
                    continue
                # All other properties are implicitly skipped due to tracking only the new desired properties

            processed_points.append(' '.join(new_parts))
            
        except (ValueError, IndexError) as e:
            # Skip malformed lines
            print(f"Skipping malformed data line: {e}")
            
    # --- 4. Final Header Assembly and Writing ---
    
    final_vertex_count = len(processed_points)
    
    # Replace the 'element vertex' line with the new count
    final_header = []
    for line in new_header:
        if line.startswith('element vertex'):
            final_header.append(f"element vertex {final_vertex_count}")
        else:
            final_header.append(line)

    try:
        with open(output_filename, 'w') as outfile:
            # Write Header
            for line in final_header:
                outfile.write(line + '\n')
            
            # Write Data
            for point_line in processed_points:
                outfile.write(point_line + '\n')

        print("-" * 50)
        print("✅ Processing complete!")
        print(f"  Total points in original file: {len(points)}")
        print(f"  Points removed (Duplicates): {points_removed_duplicate}")
        print(f"  Points removed (Classification={classification_to_remove}): {points_removed_classification}")
        print(f"  Total points in new file: **{final_vertex_count}**")
        print(f"New file saved as: **{output_filename}**")

    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


# --- CONFIGURATION ---
INPUT_FILE = 'A1-A10_ongoing6.ply' # Change this to your actual file name
OUTPUT_FILE = 'A1-A10_ongoing7.ply' 
CLASSIFICATION_TO_REMOVE = 12.0 # The value you want to filter out

# Run the function
# IMPORTANT: Ensure 'input.ply' is in the same directory as this script.
process_ply_file(INPUT_FILE, OUTPUT_FILE, CLASSIFICATION_TO_REMOVE)

Reading from: A1-A10_ongoing6.ply
Writing to: A1-A10_ongoing7.ply
Removing points with scalar_Classification = 12.0...


In [ ]:
import os

def filter_and_retype_ply(input_filename, output_filename, value_to_remove=12.0):
    """
    Reads an ASCII PLY file, removes points with a specific 
    'scalar_Classification' value, and changes the property type 
    in the header from float to int.
    """
    
    print(f"Reading from: {input_filename}")
    print(f"Writing to: {output_filename}")
    print(f"Removing points where scalar_Classification is {value_to_remove}...")
    
    if not os.path.exists(input_filename):
        print(f"Error: Input file not found at '{input_filename}'")
        return

    # Indices assumed from your header structure:
    # x: 0, y: 1, z: 2, scalar_Classification: 3, scalar_Original_cloud_index: 4
    CLASSIFICATION_INDEX = 3
    
    header_lines = []
    vertex_data_lines = []
    in_header = True
    original_vertex_count = 0
    
    try:
        # --- 1. Read and Process the Input File ---
        with open(input_filename, 'r') as infile:
            for line in infile:
                line = line.strip()
                if not line: # Skip empty lines
                    continue
                
                if in_header:
                    header_lines.append(line)
                    if line == 'end_header':
                        in_header = False
                else:
                    original_vertex_count += 1
                    parts = line.split()
                    
                    if len(parts) > CLASSIFICATION_INDEX:
                        try:
                            current_classification = float(parts[CLASSIFICATION_INDEX])
                            
                            # --- Filter Step: Keep only if classification is NOT the value to remove ---
                            if current_classification != value_to_remove:
                                # --- Retype Step: Change classification to int format for writing ---
                                parts[CLASSIFICATION_INDEX] = str(int(current_classification))
                                vertex_data_lines.append(' '.join(parts))
                                
                        except ValueError:
                            # If parsing fails, treat it as non-vertex data and skip filtering/retyping
                            vertex_data_lines.append(line)
                    else:
                        # Keep lines that aren't long enough to be standard vertex data
                        vertex_data_lines.append(line)


        # --- 2. Modify Header and Prepare Output ---
        modified_header = []
        new_vertex_count = len(vertex_data_lines)
        
        for line in header_lines:
            if line.startswith('element vertex'):
                # Update the vertex count
                modified_header.append(f'element vertex {new_vertex_count}')
            elif line.startswith('property float scalar_Classification'):
                # Change the type from float to int
                modified_header.append('property int scalar_Classification')
            else:
                # Keep other header lines as they are
                modified_header.append(line)

        # --- 3. Write the Output File ---
        with open(output_filename, 'w') as outfile:
            # Write the modified header
            outfile.write('\n'.join(modified_header) + '\n')
            
            # Write the filtered and retyped vertex data
            outfile.write('\n'.join(vertex_data_lines) + '\n')

        # --- 4. Summary ---
        points_removed = original_vertex_count - new_vertex_count
        
        print("-" * 45)
        print(f"✅ Processing complete!")
        print(f"Original points: {original_vertex_count}")
        print(f"Points removed (Classification=12.0): **{points_removed}**")
        print(f"Final points count: {new_vertex_count}")
        print(f"New file saved as: **{output_filename}**")
        print("Note: scalar_Classification property type is now 'int' in the header.")

    except Exception as e:
        print(f"An error occurred: {e}")

# --- CONFIGURATION ---
# IMPORTANT: Change 'input.ply' to the actual name of your file
INPUT_FILE = 'A1-A10_ongoing6.ply' 
OUTPUT_FILE = 'A1-A10_ongoing7.ply' 

# Run the function
filter_and_retype_ply(INPUT_FILE, OUTPUT_FILE, value_to_remove=12.0)